In [10]:
import pandas as pd
import seaborn as sns
import numpy as np
import xgboost as xgb
from tqdm import tqdm
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pranav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [86]:
from keras.models import sequential
from keras.layers.recurrent import LSTM,GRU
from keras.layers.core import Dense,Dropout,Activation
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing,decomposition,model_selection,metrics,pipeline,feature_extraction
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD



In [54]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")

In [19]:
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [55]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, train.target.values, 
                                                  stratify=train.target.values, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [65]:
import pickle

In [66]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

xtrain_tfidf, tfidf_vectorizer = tfidf(xtrain)
xvalid_tfidf = tfidf_vectorizer.transform(xvalid)
pickle.dump(tfidf_vectorizer, open("tfidf1.pkl", "wb"))


#tfv.fit(list(xtrain) + list(xvalid))
#xtrain_tfv =  tfv.transform(xtrain) 
#xvalid_tfv = tfv.transform(xvalid)

In [57]:
from sklearn.metrics import f1_score

In [67]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfidf, ytrain)
predictions = clf.predict(xvalid_tfidf)
print("f1 score : {}".format(f1_score(yvalid,predictions)))

f1 score : 0.7636363636363637


In [59]:
predictions.shape


(762,)

In [28]:
# Trying another model

In [77]:
# Count Vectorizer

count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train["text"][0:5])

In [78]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [80]:
train_vectors = count_vectorizer.fit_transform(train["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test["text"])

In [87]:
clf = RidgeClassifier()
#LogisticRegression(C = 1)
scores = model_selection.cross_val_score(clf, train_vectors, train["target"], cv=3, scoring="f1")
scores



array([0.59421842, 0.56455572, 0.64149093])

In [89]:
clf.fit(train_vectors, train["target"])
sub["target"] = clf.predict(test_vectors)

In [90]:
sub.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [91]:
sub.to_csv("sub1.csv", index=False)